In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dtuimldmtools as dtu
from scipy import stats
import itertools

# Load and preprocess dataset again

In [ ]:
data_path = "data/"
seeds_dataset = "seeds_dataset.txt"
dataset_file = data_path + seeds_dataset

In [ ]:
data = np.loadtxt(dataset_file)
# Validate shape of the dataset, 210 rows with 8 attributes
data.shape

(210, 8)

In [49]:
X = data
# attributeNames are not present in the dataset, just gonna hardcode based on the website
attributeNames = [
    "area_A",
    "perimeter_P",
    "compactness_C",
    "length_of_kernel",
    "width_of_kernel",
    "asymmetry_coefficient",
    "length_of_kernel_groove",
    "class",
]
N = data.shape[0]
M = data.shape[1]
y = X[:, -1]
# This is derived from the website
classNames = ["Kama", "Rosa", "Canadian"]
C = len(classNames)
attributeNames, N, M, y, y.shape, classNames, C

(['area_A',
  'perimeter_P',
  'compactness_C',
  'length_of_kernel',
  'width_of_kernel',
  'asymmetry_coefficient',
  'length_of_kernel_groove',
  'class'],
 210,
 8,
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,

### Ensure zero-indexing

In [50]:
X[:, -1] -= 1
X.shape, X[:, -1]

((210, 8),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2.]))

### Remove outlier as shown from project_1

In [51]:
attribute_index = attributeNames.index("length_of_kernel")
lowest_index = np.argmin(X[:, 3])
X_updated = np.delete(X, lowest_index, axis=0)
y = np.delete(y, lowest_index, axis=0)
N -= 1
N, X_updated.shape, y.shape

(209, (209, 8), (209,))

### Remove class column because we would not need it for classification

In [52]:
X_updated = X_updated[:, :-1]
X_updated.shape

(209, 7)

### Standardize data
Data standardization/ data scaling needs to be done if the data have huge or scattered values, machine learning model needs smaller and coherent values. Data scaling, standardize values in the data set for better results."

https://www.kaggle.com/discussions/questions-and-answers/159183#910328

In [ ]:
# Standardize the data
X_mean = np.mean(X_updated, axis=0)
X_std = np.std(X_updated, axis=0)
X_standardized = (X_updated - X_mean) / X_std

In [54]:
X_standardized.shape, y.shape

((209, 7), (209,))

# Regression

In [55]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.stats as st
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Load dataset
# data = np.loadtxt("seeds_dataset.txt")

# Predict 'Area' (first column), use the rest as features
y = data[:, 0]
X = data[:, 1:7]  # Exclude Area

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Two-level cross-validation parameters
K1 = K2 = 10
hidden_units = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
lambdas = np.power(10.0, range(-5, 9
))  # Regression λ values: 1e-5 to 1e5

# ANN model training function
def train_model(X_train, y_train, X_val, y_val, hidden_units, lr=0.01, max_epochs=1000):
    M = X_train.shape[1]
    model = nn.Sequential(
        nn.Linear(M, hidden_units),
        nn.Tanh(),
        nn.Linear(hidden_units, 1)
    )
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(max_epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        y_pred = model(X_val)
        mse = mean_squared_error(y_val.numpy(), y_pred.numpy())
    return model, mse

# Statistical evaluation
def statistically_evaluate(name1, squared_error_1, name2, squared_error_2):
    alpha = 0.05
    z = squared_error_1 - squared_error_2
    CI = st.t.interval(1 - alpha, len(z) - 1, loc=np.mean(z), scale=st.sem(z))
    p = 2 * st.t.cdf(-np.abs(np.mean(z)) / st.sem(z), df=len(z) - 1)
    print(f"Confidence interval of {name1}-{name2}: {CI}")
    print(f"p-value: {p}\n")
    return np.mean(z), CI[0], CI[1], p

# Results container
results = []

outer_cv = KFold(n_splits=K1, shuffle=True, random_state=1)

for outer_fold, (train_idx, test_idx) in enumerate(outer_cv.split(X)):
    print(f"Outer fold {outer_fold + 1}/{K1}")
    X_par, y_par = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]

    inner_cv = KFold(n_splits=K2, shuffle=True, random_state=outer_fold)

    # --- ANN ---
    best_ann_err = np.inf
    best_h = None
    best_ann_model = None

    for h in hidden_units:
        ann_errs = []
        for inner_train_idx, val_idx in inner_cv.split(X_par):
            X_train = torch.tensor(X_par[inner_train_idx], dtype=torch.float32)
            y_train = torch.tensor(y_par[inner_train_idx].reshape(-1, 1), dtype=torch.float32)
            X_val = torch.tensor(X_par[val_idx], dtype=torch.float32)
            y_val = torch.tensor(y_par[val_idx].reshape(-1, 1), dtype=torch.float32)
            model, mse = train_model(X_train, y_train, X_val, y_val, hidden_units=h)
            ann_errs.append(mse)
        mean_ann_err = np.mean(ann_errs)
        if mean_ann_err < best_ann_err:
            best_ann_err = mean_ann_err
            best_h = h
            X_par_tensor = torch.tensor(X_par, dtype=torch.float32)
            y_par_tensor = torch.tensor(y_par.reshape(-1, 1), dtype=torch.float32)
            best_ann_model, _ = train_model(X_par_tensor, y_par_tensor, X_par_tensor, y_par_tensor, hidden_units=h)

    # --- Regression (Ridge) ---
    best_regression_err = np.inf
    best_lambda = None
    best_regression_model = None

    for lam in lambdas:
        regression_errs = []
        for inner_train_idx, val_idx in inner_cv.split(X_par):
            X_train, y_train = X_par[inner_train_idx], y_par[inner_train_idx]
            X_val, y_val = X_par[val_idx], y_par[val_idx]
            model = Ridge(alpha=lam)
            model.fit(X_train, y_train)
            y_val_pred = model.predict(X_val)
            regression_errs.append(mean_squared_error(y_val, y_val_pred))
        mean_regression_err = np.mean(regression_errs)
        if mean_regression_err < best_regression_err:
            best_regression_err = mean_regression_err
            best_lambda = lam
            best_regression_model = Ridge(alpha=lam).fit(X_par, y_par)

    # --- Evaluation ---
    y_pred_baseline = np.full_like(y_test, np.mean(y_par))
    y_pred_regression = best_regression_model.predict(X_test)

    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    best_ann_model.eval()
    with torch.no_grad():
        y_pred_ann = best_ann_model(X_test_tensor).numpy()

    ann_test_error = mean_squared_error(y_test, y_pred_ann)
    regression_test_error = mean_squared_error(y_test, y_pred_regression)
    baseline_test_error = mean_squared_error(y_test, y_pred_baseline)

    results.append([outer_fold + 1, best_h, ann_test_error, best_lambda, regression_test_error, baseline_test_error])


Outer fold 1/10
Outer fold 2/10
Outer fold 3/10
Outer fold 4/10
Outer fold 5/10
Outer fold 6/10
Outer fold 7/10
Outer fold 8/10
Outer fold 9/10
Outer fold 10/10


In [56]:

# Create DataFrame
df_results = pd.DataFrame(results, columns=[
    "Outer fold", "h*", "ann E_test", "lambda*", "regression E_test", "baseline E_test"
])
print("\nFinal cross-validation results:")
#print(df_results)
df_results


Final cross-validation results:


,Outer fold,h*,ann E_test,lambda*,regression E_test,baseline E_test
0,1,21,10.379972,0.01,0.015133,6.275958
1,2,7,0.099721,0.01,0.014889,10.159598
2,3,13,0.054595,0.10,0.010238,10.798736
3,4,19,0.004206,0.10,0.005802,6.842417
4,5,15,0.085733,0.10,0.019104,12.788180
5,6,9,0.084067,0.10,0.008942,8.954705
6,7,9,0.570766,0.01,0.013297,9.717176
7,8,19,0.013464,0.10,0.009461,5.697848
8,9,11,0.401424,0.01,0.015283,8.068414
9,10,19,0.010761,0.10,0.021016,5.481287


In [57]:
# Statistical evaluation table
diff_results = []
comparisons = [
    ("ann vs regr", df_results["ann E_test"].values, df_results["regression E_test"].values),
    ("ann vs base", df_results["ann E_test"].values, df_results["baseline E_test"].values),
    ("regr vs base", df_results["regression E_test"].values, df_results["baseline E_test"].values)
]

for label, err1, err2 in comparisons:
    z = err1 - err2
    ci = st.t.interval(0.95, len(z) - 1, loc=np.mean(z), scale=st.sem(z))
    p = 2 * st.t.cdf(-np.abs(np.mean(z)) / st.sem(z), df=len(z) - 1)
    diff_results.append([label, p, ci[0], ci[1]])

df_stat_eval = pd.DataFrame(diff_results, columns=[
    "comparison", "p-value", "lower bound", "upper bound"
])


print("\nStatistical evaluation summary:")
#print(df_stat_eval)
df_stat_eval


Statistical evaluation summary:


,comparison,p-value,lower bound,upper bound
0,ann vs regr,0.288030,-1.161101,3.475410
1,ann vs base,0.000721,-10.602554,-4.013368
2,regr vs base,0.000002,-10.199907,-6.730324


# Classification
3 models would be implemented
* Baseline model: majority model
* Logistic regression with a softmax activation function at the end
* ANN

## K fold cross validation

In [58]:
from sklearn import model_selection

In [59]:
K = 10
CV = model_selection.KFold(K, shuffle=True)

## Baseline model
Majority class classifier : where the most frequent class in the data is predicted for all observations. For instance, if we have 80% of observations in class A and 20% in class B for a binary classification problem, the baseline model would predict Class A for all instances.
https://medium.com/@preethi_prakash/understanding-baseline-models-in-machine-learning-3ed94f03d645

In [60]:
from sklearn.dummy import DummyClassifier

In [61]:
unique, counts = np.unique(y, return_counts=True)
unique, counts

(array([10.59, 10.74, 10.79, 10.8 , 10.82, 10.83, 10.91, 10.93, 11.02,
        11.14, 11.18, 11.19, 11.21, 11.23, 11.24, 11.26, 11.27, 11.34,
        11.35, 11.36, 11.4 , 11.41, 11.42, 11.43, 11.48, 11.49, 11.55,
        11.56, 11.65, 11.75, 11.81, 11.82, 11.83, 11.84, 11.87, 12.01,
        12.02, 12.05, 12.08, 12.1 , 12.11, 12.13, 12.15, 12.19, 12.21,
        12.22, 12.26, 12.3 , 12.36, 12.37, 12.38, 12.44, 12.46, 12.49,
        12.54, 12.55, 12.62, 12.67, 12.7 , 12.72, 12.73, 12.74, 12.76,
        12.78, 12.79, 12.8 , 12.88, 12.89, 13.02, 13.07, 13.16, 13.2 ,
        13.22, 13.32, 13.34, 13.37, 13.45, 13.5 , 13.54, 13.74, 13.78,
        13.8 , 13.84, 13.89, 13.94, 13.99, 14.01, 14.03, 14.09, 14.11,
        14.16, 14.28, 14.29, 14.33, 14.34, 14.37, 14.38, 14.43, 14.46,
        14.49, 14.52, 14.59, 14.69, 14.7 , 14.79, 14.8 , 14.86, 14.88,
        14.92, 14.99, 15.01, 15.03, 15.05, 15.11, 15.26, 15.36, 15.38,
        15.49, 15.5 , 15.56, 15.57, 15.6 , 15.69, 15.78, 15.88, 15.99,
      

In [62]:
baseline_error_rates = []
for train_index, test_index in CV.split(X_standardized, y):
    X_train = X_standardized[train_index]
    y_train = y[train_index]
    X_test = X_standardized[test_index]
    y_test = y[test_index]
    baseline = DummyClassifier(strategy='most_frequent')
    baseline.fit(X_train, y_train)
    y_preds = baseline.predict(X_test)

    e = y_preds != y_test
    error_rate = sum(e) / len(e)
    baseline_error_rates.append(error_rate)
    print(
        f"Number of miss-classifications for baseline model:\n\t {sum(e)} out of {len(e)}. Overall error_rate {error_rate}"
    )
mean_error_rate = np.mean(np.asarray(baseline_error_rates))
print(f"mean_error_rate for baseline model is {mean_error_rate}")

ValueError: Found input variables with inconsistent numbers of samples: [209, 210]

## Logistic regression
Add an extra variable lambda to penalise large weights
Testing the range of lambda from 10^-5 to 10^4

In [ ]:
lambdas = np.power(10.0, range(-5, 5))
lambdas.shape, lambdas

In [ ]:
from dtuimldmtools import rlr_validate
from scipy.special import softmax

In [ ]:
error_rates_and_lambda = []
yhat_log = []
y_true_log = []
lambdas = np.power(10.0, range(-5, 5))
for train_index, test_index in CV.split(X_standardized, y):
    X_train = X_standardized[train_index]
    y_train = y[train_index].astype(np.int64)
    X_test = X_standardized[test_index]
    y_test = y[test_index].astype(np.int64)
    internal_cross_validation = 10
    input_features = M - 1
    # One-hot encoding
    Y_train = np.zeros((len(y_train), C))
    for i, label in enumerate(y_train):
        Y_train[i, label] = 1
    # Function returns:
    # MSE averaged over 'cvf' folds,
    # optimal value of lambda,
    # average weight values for all lambdas,
    # MSE train&validation errors for all lambdas.
    # The cross validation splits are standardized based on the mean and standard deviation of the training set when estimating the regularization strength.
    _, opt_lambda, _, _, _, = rlr_validate(X_train, y_train, lambdas, internal_cross_validation)
    Xty = X_train.T @ Y_train
    XtX = X_train.T @ X_train
    # Estimate weights for the optimal value of lambda, on entire training set
    lambdaI = opt_lambda * np.eye(input_features)
    lambdaI[0, 0] = 0  # Do no regularize the bias term
    # Recall: Introduce regularization term λ‖w‖2 to penalize large weights, remove the significance of these weight
    # Recall: (X^T@X + lambdaI) @ w = X^T @ y
    estimated_weights = np.linalg.solve(XtX + lambdaI, Xty).squeeze()
    prediction_logits = X_test @ estimated_weights
    predicted_class = np.argmax(softmax(prediction_logits), axis=1)
    yhat_log.append(predicted_class)
    y_true_log.append(y_test)
    e = predicted_class != y_test
    error_rate = sum(e) / len(e)
    error_rates_and_lambda.append((error_rate, opt_lambda))
    print(
        f"Number of miss-classifications for logic regression model with optimal lambda value {opt_lambda}:\n\t {sum(e)} out of {len(e)}. Overall error_rate {error_rate}"
    )

error_rates_and_lambda, len(error_rates_and_lambda)
yhat_log = np.concatenate(yhat_log)
y_true_log = np.concatenate(y_true_log)
yhat_log.shape, y_true_log.shape

## Multiclass ANN
Since we have three distinct classes: Kama, Rosa and Canadian, we adopt a multiclass approach. 
As complexity-controlling parameter
for the ANN, we will use the number of hidden units3 h. Based on a few test-runs, select
a reasonable range of values for h (which should include h = 1), and describe the range of
values you will use for h

In [ ]:
import torch
from torch import nn
from dtuimldmtools import dbplotf, train_neural_net, visualize_decision_boundary

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
!nvidia-smi

In [ ]:
device

In [ ]:
error_rates = []
for train_index, test_index in CV.split(X_standardized, y):
    X_train = torch.from_numpy(X_standardized[train_index]).type(torch.float)
    y_train = torch.from_numpy(y[train_index]).type(torch.long)
    X_test = torch.from_numpy(X_standardized[test_index]).type(torch.float)
    y_test = torch.from_numpy(y[test_index]).type(torch.long)

### ANN training and mean_error

In [ ]:
from operator import itemgetter

In [ ]:
hidden_units_range = 10
initial_hidden_units = 2
error_rates_and_hidden_units_in_folds = []
for train_index, test_index in CV.split(X_standardized, y):
    X_train = torch.from_numpy(X_standardized[train_index]).type(torch.float)
    y_train = torch.from_numpy(y[train_index]).type(torch.long)
    X_test = torch.from_numpy(X_standardized[test_index]).type(torch.float)
    y_test = torch.from_numpy(y[test_index]).type(torch.long)
    error_rates_and_hidden_units = []
    for n_hidden_units in range(
        initial_hidden_units, initial_hidden_units + hidden_units_range
    ):
        # in the actual code, we would vary the number of hidden_units here
        # e.g. for i in range (100) -> calculate mean_error_rate
        # Recall that last column represents the classes and should not be used as an input feature
        input_features = M - 1
        num_epochs = 300
        loss_fn = torch.nn.CrossEntropyLoss()
        seed_model = lambda: torch.nn.Sequential(
            torch.nn.Linear(input_features, n_hidden_units),
            torch.nn.ReLU(),
            torch.nn.Linear(n_hidden_units, C),
            torch.nn.Softmax(dim=1),
        )
        net, final_loss, learning_curve = train_neural_net(
            seed_model,
            loss_fn,
            X=X_train,
            y=y_train,
            n_replicates=3,
            max_iter=num_epochs,
        )
        print("\n\t model loss: {}\n".format(final_loss))

        # Determine probability of each class using trained network
        softmax_logits = net(X_test)
        # convert to label with the highest probability# Parametergrids
lambda_values = np.logspace(-5, 2, 10)
hidden_units = [1, 2, 4, 8, 16]

# Outer K-fold
K1 = 10
outer_cv = KFold(n_splits=K1, shuffle=True, random_state=1)

# Ergebnisse speichern
results = []
y_pred = torch.argmax(softmax_logits, dim=1)
# Compare error against ground truth y_test
e = y_pred != y_test
error_rate = sum(e) / len(e)
error_rates_and_hidden_units.append((error_rate, n_hidden_units))
print(
    f"Number of miss-classifications for ANN:\n\t {sum(e)} out of {len(e)}. Overall error_rate {error_rate}"
)

smallest_error_rate, num_hidden_units = min(
error_rates_and_hidden_units, key=itemgetter(0)
)
error_rates_and_hidden_units_in_folds.append(
(smallest_error_rate, num_hidden_units)
)
print(
f"smallest_error_rate for {num_hidden_units} hidden_units is {smallest_error_rate}"
)



## Plot comparison table between the 3 models

In [ ]:
baseline_error_rates

In [ ]:
error_rates_and_lambda

In [ ]:
error_rates_and_hidden_units_in_folds

In [ ]:
from torch import tensor

error_rates_from_lambda = [rate for rate, _ in error_rates_and_lambda]
lambda_values = [lmbda for _, lmbda in error_rates_and_lambda]

error_rates_from_hidden_units = [
    rate.item() if hasattr(rate, "item") else float(rate)
    for rate, _ in error_rates_and_hidden_units_in_folds
]
hidden_units = [units for _, units in error_rates_and_hidden_units_in_folds]
error_rates_from_lambda, error_rates_from_hidden_units

## 

In [ ]:
import matplotlib.pyplot as plt

num_folds = len(baseline_error_rates)

table_data = [
    [
        fold + 1,
        hidden_units[fold],
        error_rates_from_hidden_units[fold],
        lambda_values[fold],
        error_rates_from_lambda[fold],
        baseline_error_rates[fold],
    ]
    for fold in range(num_folds)
]

# Column headers (as shown in the image)
col_labels = ["i", "x_t^*", "E_t^NN", "λ_t^*", "E_t^λ", "E_t^baseline"]

# Create the plot
fig, ax = plt.subplots()
ax.axis("off")

# Plot the table
table = ax.table(
    cellText=table_data, colLabels=col_labels, loc="center", cellLoc="center"
)
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(3, 3)

plt.show()

# Evaluation of models using mc neymar test
![mc_neymar](museum_of_poor/mc_neymar.png)
Perform a statistical evaluation of your three models similar to the previous section. That
is, compare the three models pairwise. We will once more allow some freedom in what test
to choose. Therefore, choose either:
setup I (section 11.3): Use McNemar’s test described in Box 11.3.2)
setup II (section 11.4): Use the method described in Box 11.4.1)
Include p-values and confidence intervals for the three pairwise tests in your report and
conclude on the results: Is one model better than the other? Are the two models better
than the baseline? Are some of the models identical? What recommendations would you
make based on what you’ve learned?

### To ensure a fair comparison, we need to perform the train and test on the same split

In [ ]:
yhat_dummy = []
yhat_log = []
yhat_ann = []
y_true = []
lambdas = np.power(10.0, range(-5, 5))
for train_index, test_index in CV.split(X_standardized, y):
    X_train = X_standardized[train_index]
    y_train = y[train_index].astype(np.int64)
    X_test = X_standardized[test_index]
    y_test = y[test_index].astype(np.int64)
    internal_cross_validation = 10
    input_features = M - 1
    y_true.append(y_test)

    # One-hot encoding
    Y_train_hot = np.zeros((len(y_train), C))
    for i, label in enumerate(y_train):
        Y_train_hot[i, label] = 1

    # Dummy classifier
    baseline = DummyClassifier(strategy='most_frequent')
    baseline.fit(X_train, y_train)
    y_dummy_pred = baseline.predict(X_test)
    yhat_dummy.append(y_dummy_pred)
    (
        _,
        opt_lambda,
        _,
        _,
        _,
    ) = rlr_validate(X_train, y_train, lambdas, internal_cross_validation)
    Xty = X_train.T @ Y_train_hot
    XtX = X_train.T @ X_train
    # Estimate weights for the optimal value of lambda, on entire training set
    lambdaI = opt_lambda * np.eye(input_features)
    lambdaI[0, 0] = 0  # Do no regularize the bias term
    # Recall: Introduce regularization term λ‖w‖2 to penalize large weights, remove the significance of these weight
    # Recall: (X^T@X + lambdaI) @ w = X^T @ y
    estimated_weights = np.linalg.solve(XtX + lambdaI, Xty).squeeze()
    prediction_logits = X_test @ estimated_weights
    predicted_class = np.argmax(softmax(prediction_logits), axis=1)
    yhat_log.append(predicted_class)

    X_train_tensor = torch.from_numpy(X_standardized[train_index]).type(torch.float)
    y_train_tensor = torch.from_numpy(y[train_index]).type(torch.long)
    X_test_tensor = torch.from_numpy(X_standardized[test_index]).type(torch.float)
    y_test_tensor = torch.from_numpy(y[test_index]).type(torch.long)
    error_rates_and_hidden_units = []
    input_features = M - 1
    num_epochs = 300
    loss_fn = torch.nn.CrossEntropyLoss()
    seed_model = lambda: torch.nn.Sequential(
        torch.nn.Linear(input_features, n_hidden_units),
        torch.nn.ReLU(),
        torch.nn.Linear(n_hidden_units, C),
        torch.nn.Softmax(dim=1),
    )
    for n_hidden_units in range(
        initial_hidden_units, initial_hidden_units + hidden_units_range
    ):
        # in the actual code, we would vary the number of hidden_units here
        # e.g. for i in range (100) -> calculate mean_error_rate
        # Recall that last column represents the classes and should not be used as an input feature

        net, final_loss, learning_curve = train_neural_net(
            seed_model,
            loss_fn,
            X=X_train_tensor,
            y=y_train_tensor,
            n_replicates=3,
            max_iter=num_epochs,
        )

        # Determine probability of each class using trained network
        softmax_logits = net(X_test_tensor)
        # convert to label with the highest probability
        y_preds = torch.argmax(softmax_logits, dim=1)
        # Compare error against ground truth y_test
        e = y_preds != y_test
        error_rate = sum(e) / len(e)
        error_rates_and_hidden_units.append((error_rate, n_hidden_units, y_pred))


    _, _, best_pred = min(
        error_rates_and_hidden_units, key=itemgetter(0)
    )
    yhat_ann.append(best_pred)
yhat_dummy = np.concatenate(yhat_dummy)
yhat_log = np.concatenate(yhat_log)
yhat_ann= np.concatenate(yhat_ann)
y_true = np.concatenate(y_true)
yhat_dummy.shape, yhat_log.shape, y_true.shape

In [ ]:
from dtuimldmtools import mcnemar

# Compute the Jeffreys interval
alpha = 0.05
[thetahat, CI, p] = mcnemar(y_true_dummy, yhat[:, 0], yhat[:, 1], alpha=alpha)

print("theta = theta_A-theta_B point estimate", thetahat, " CI: ", CI, "p-value", p)

## Final regression model with $\lambda$ = 0.01

In [ ]:
error_rates = []
for train_index, test_index in CV.split(X_standardized, y):
    X_train = X_standardized[train_index]
    y_train = y[train_index].astype(np.int64)
    X_test = X_standardized[test_index]
    y_test = y[test_index].astype(np.int64)
    internal_cross_validation = 10
    input_features = M - 1
    # One-hot encoding
    Y_train = np.zeros((len(y_train), C))
    for i, label in enumerate(y_train):
        Y_train[i, label] = 1
    opt_lambda = 0.01
    Xty = X_train.T @ Y_train
    XtX = X_train.T @ X_train
    # Estimate weights for the optimal value of lambda, on entire training set
    lambdaI = opt_lambda * np.eye(input_features)
    lambdaI[0, 0] = 0  # Do no regularize the bias term
    # Recall: Introduce regularization term λ‖w‖2 to penalize large weights, remove the significance of these weight
    # Recall: (X^T@X + lambdaI) @ w = X^T @ y
    estimated_weights = np.linalg.solve(XtX + lambdaI, Xty).squeeze()
    prediction_logits = X_test @ estimated_weights
    predicted_classes = np.argmax(softmax(prediction_logits), axis=1)
    e = predicted_classes != y_test
    error_rate = sum(e) / len(e)
    error_rates.append(error_rate)
    print(
        f"Number of miss-classifications for logic regression model with optimal lambda value {opt_lambda}:\n\t {sum(e)} out of {len(e)}. Overall error_rate {error_rate}"
    )
error_rates, len(error_rates)